## **IMPORTING DATASETS FROM KAGGLE**

In [ ]:
# Importing datasets from Kaggle

!pip install -q kaggle

In [ ]:
from google.colab import files

files.upload()

In [ ]:
!mkdir ~/.kaggle

In [ ]:
!cp kaggle.json ~/.kaggle/

In [ ]:
!chmod 600 ~/.kaggle/kaggle.json

In [ ]:
!kaggle datasets download grassknoted/asl-alphabet --force

In [ ]:
!unzip asl-alphabet.zip

## **IMPORTING THE NECESSARY LIBRARIES**

In [ ]:
# !pip install fastai --upgrade

In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
import glob
import os
from PIL import Image
# import fastai
from fastai.vision.all import *

## **INTERPRETATION OF THE DATA**

In [ ]:
train_path = '/content/asl_alphabet_train/asl_alphabet_train/'
test_path = '/content/asl_alphabet_test/asl_alphabet_test/'
sample_image = '/content/asl_alphabet_train/asl_alphabet_train/J/J1.jpg'
img = Image.open(sample_image)
w, h = img.size
print("Width of image:", w)
print("Height of image:", h)
img

In [ ]:
db = DataBlock(blocks = (ImageBlock, CategoryBlock), get_items = get_image_files, get_y = parent_label, splitter = RandomSplitter(valid_pct = 0.2), item_tfms = Resize(128))
file = get_image_files(train_path)
print(file[1])
print(file[1].parent.name)

In [ ]:
dataloaders = db.dataloaders(train_path, bs = 32)

In [ ]:
dataloaders.show_batch()

## **TRAINING THE MODEL**

In [ ]:
model_temp = cnn_learner(dataloaders, models.resnet50, metrics = [accuracy], loss_func = CrossEntropyLossFlat())

In [ ]:
model_temp.fine_tune(5)

In [ ]:
model_temp.show_results()

## **TESTING THE MODEL**

In [ ]:
test_images_temp = get_image_files(test_path) 
test_dataloaders_temp = model_temp.dls.test_dl(test_images_temp) 
pred_temp, _ = model_temp.get_preds(dl = test_dataloaders_temp) 
preds_temp = pred_temp.argmax(dim = -1).numpy() 

In [ ]:
df_test_temp = pd.DataFrame(os.listdir(test_path)).rename(columns={0:'Image_Name'}) 
df_test_temp['Actual_Value'] = df_test_temp['Image_Name'].str.split('_').str[0]
df_test_temp['Predicted_Value'] = preds_temp
vocab = dict(zip(np.arange(29), dataloaders.vocab))
df_test_temp['Predicted_Value'] = df_test_temp['Predicted_Value'].replace(vocab)
df_test_temp['Image_Path'] = test_path + df_test_temp['Image_Name']
fig, axes = plt.subplots(nrows=4, ncols=7, figsize=(20, 12),
                        subplot_kw={'xticks': [], 'yticks': []})

for i, ax in enumerate(axes.flat):
    ax.imshow(plt.imread(df_test_temp['Image_Path'].iloc[i]))
    ax.set_title(f"True: {df_test_temp.Actual_Value.iloc[i]}\nPredicted: {df_test_temp.Predicted_Value.iloc[i]}", fontsize = 15)
plt.tight_layout()
plt.show()